In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 각각의 파일 읽어오기

In [3]:
import pandas as pd
train = pd.read_csv('../input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
test = pd.read_csv('../input/bike-sharing-demand/test.csv',parse_dates=['datetime'])
sub = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')

In [4]:
train.shape, test.shape, sub.shape

In [5]:
train.head()

In [6]:
train.columns

In [7]:
train.info()

- season : 계절
- holiday : 휴일 여부 (0,1)
- workingday : 일하는 날 여부 (0,1)
- weather : 날씨 (1,2,3,4)
- temp : 온도
- atemp : 체감 온도
- humidity : 습도
- windspeed : 풍속

In [8]:
train.describe()

## (실습1) 데이터를 알아가기 위한 여러가지 질문을 작성해 보자.
### 01. 데이터 날짜는 언제부터 언제까지 데이터일까?

In [9]:
train.datetime.min(),  train.datetime.max()

### 02. 실제 빌린 대수(count)와 다른 정보간의 관계는 어떤 관계가 있을까?
- count와 temp의 관계

In [10]:
train.corr()['count']

### 03. count와 다른 변수간의 관계 확인 - corr() 상관계수
- 가장 높은 상관관계를 갖는 순서로 정렬시켜보자.(pandas)
- 이를 수평 막대 그래프로 표시해 보자.
- x축, y축 레이블, 제목을 표시해보자

In [11]:
cnt_corr = train.corr()['count'].sort_values(ascending=False)

In [12]:
cnt_corr = pd.DataFrame(cnt_corr)
cnt_corr.T

In [13]:
import matplotlib.pyplot as plt

In [14]:
cnt_corr = cnt_corr.T

In [15]:
X= list(cnt_corr.columns)
y= cnt_corr.values.tolist()

(음수, 양수 동시에 안되니 나눠서)

In [16]:
data1 = [1.0,
  0.9709481058098266,
  0.6904135653286754,
  0.3944536449672518,
  0.38978443662697554,
  0.1634390165763605,
  0.10136947021033216,
  0.01159386609157489,0,0,0]
data2 = [0,  0,  0,  0,  0,  0,  0,  0,  -0.005392984477774549,  -0.1286552010385059,  -0.31737147887659584]
plt.barh(X, data1,color='r')
plt.barh(X, data2,color='b')
plt.show()

### 04. 계절별 데이터는 어떤 패턴을 가질까?
- season 특징(정보) 확인해 보기
- 계절별 데이터를 확인 및 시각화 해 보자.
- x축을 1,2,3,4만 표시되도록 하자.

In [17]:
train.head()

In [18]:
seasondf = train.groupby('season').mean()
seasondf

In [19]:
seasondf.plot()
plt.xticks(seasondf.index)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

### 05. 쉬는날과 쉬는 날이 아닌 데이터는 어떤 패턴을 가질까?
holiday의 값의 종류와 count를 확인해 보기

In [20]:
train.head(2)

In [21]:
train.groupby(['datetime','holiday']).mean()

In [33]:
holiday_0 = train[train['holiday']==0]
holiday_0 = holiday_0.drop('season', axis=1)
holiday_0 = holiday_0.drop('holiday', axis=1)
holiday_0 = holiday_0.groupby('datetime').mean()

holiday_1 = train[train['holiday']==1]
holiday_1 = holiday_1.drop('season', axis=1)
holiday_1 = holiday_1.drop('holiday', axis=1)
holiday_1 = holiday_1.groupby('datetime').mean()

In [34]:
holiday_0.plot(kind='line')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
holiday_1.plot(kind='line')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

### 06. weather는 어떤 값을 갖고, 각각의 데이터의 수는 얼마나 될까?
weather의 값의 종류와 count를 확인해 보기

In [35]:
train['weather'].unique()

In [42]:
train['weather'].value_counts()

### 06. 아래의 값의 분포 2행 2열의 그래프
- temp, atemp, humidity, windspeed
- (suptitle, 크기(size)=20) ) 
- x축 레이블(크기 17)

In [51]:
sixthdf = train[['datetime','temp','atemp','humidity','windspeed']]

In [52]:
sixthdf.set_index('datetime', inplace=True)

In [56]:
sixthdf.plot(subplots=True, layout=(2,2))

In [72]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('bike_demand', fontsize=20)
fig.subplots_adjust(hspace=0.5)

plt.subplot(2,2,1)
sixthdf.temp.plot(color ='r')
plt.title('temp')
plt.xlabel('datetime', fontsize=17)


plt.subplot(2,2,2)
sixthdf.atemp.plot(color ='g')
plt.title('atemp')
plt.xlabel('datetime', fontsize=17)

plt.subplot(2,2,3)
sixthdf.humidity.plot(color ='b')
plt.title('humidity')
plt.xlabel('datetime', fontsize=17)

plt.subplot(2,2,4)
sixthdf.windspeed.plot(color ='m')
plt.title('windspeed')
plt.xlabel('datetime', fontsize=17)

### 07. 여러 특징(피처)의 값들의 분포는 어떠할까?   => 잘 이해가 안됩니다
- temp의 값의 분포는 어떠할까?
- atemp의 값의 분포는 어떠할까?
- humidity의 값의 분포는 어떠할까?
- windspeed의 값의 분포는 어떠할까?
- 전체 그래프에 대한 제목을 달아보자(suptitle, 크기(size)=20) )
- 각각의 그래프에 대한 x축 레이블을 넣어보자(크기는 17)